<a href="https://colab.research.google.com/github/bintangnabiil/Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/blob/main/Rangkuman_Chapter_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 17: Representation Learning dan Generative Learning menggunakan Autoencoders
Autoencoder adalah jenis neural network yang dilatih untuk merekonstruksi inputnya. Struktur dasar autoencoder terdiri dari dua bagian utama: encoder yang mengkompresi input menjadi representasi laten (latent representation), dan decoder yang merekonstruksi output dari representasi laten tersebut. Autoencoder memiliki banyak aplikasi praktis dalam machine learning, mulai dari dimensionality reduction, feature learning, hingga generative modeling.
<br><br>

##Arsitektur Autoencoder
Autoencoder memiliki arsitektur yang simetris dengan bottleneck di tengah:

- Input Layer: Menerima data original
- Encoder: Mengkompresi input menjadi representasi laten yang lebih kecil
- Latent Space (Bottleneck): Representasi terkompresi dari input
- Decoder: Merekonstruksi data dari representasi laten
- Output Layer: Menghasilkan rekonstruksi yang seharusnya mirip dengan input
<br><br>

##Fungsi Loss
Autoencoder dilatih dengan meminimalkan reconstruction loss, yaitu perbedaan antara input original dan output yang direkonstruksi. Untuk data kontinu, biasanya menggunakan Mean Squared Error (MSE), sedangkan untuk data binary menggunakan binary cross-entropy.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Contoh implementasi autoencoder sederhana
def create_simple_autoencoder(input_dim, encoding_dim):
    # Encoder
    input_layer = keras.layers.Input(shape=(input_dim,))
    encoded = keras.layers.Dense(encoding_dim, activation='relu')(input_layer)

    # Decoder
    decoded = keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

    # Autoencoder model
    autoencoder = keras.Model(input_layer, decoded)

    # Encoder model (untuk mendapatkan representasi laten)
    encoder = keras.Model(input_layer, encoded)

    # Decoder model
    encoded_input = keras.layers.Input(shape=(encoding_dim,))
    decoder_layer = autoencoder.layers[-1]
    decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

    return autoencoder, encoder, decoder

# Compile model
autoencoder, encoder, decoder = create_simple_autoencoder(784, 32)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

##Jenis-jenis Autoencoder
###1. Undercomplete Autoencoder
Undercomplete autoencoder memiliki dimensi latent space yang lebih kecil dari input. Ini memaksa model untuk belajar representasi yang paling penting dari data, sehingga berfungsi sebagai dimensionality reduction.

In [ ]:
# Implementasi Undercomplete Autoencoder untuk MNIST
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load dan preprocess data MNIST
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Buat undercomplete autoencoder
input_dim = 784  # 28x28 pixels
encoding_dims = [128, 64, 32]  # Progressively smaller

def create_deep_autoencoder(input_dim, encoding_dims):
    input_layer = keras.layers.Input(shape=(input_dim,))

    # Encoder
    x = input_layer
    for dim in encoding_dims:
        x = keras.layers.Dense(dim, activation='relu')(x)

    encoded = x

    # Decoder
    for dim in reversed(encoding_dims[:-1]):
        x = keras.layers.Dense(dim, activation='relu')(x)

    decoded = keras.layers.Dense(input_dim, activation='sigmoid')(x)

    autoencoder = keras.Model(input_layer, decoded)
    encoder = keras.Model(input_layer, encoded)

    return autoencoder, encoder

autoencoder, encoder = create_deep_autoencoder(784, [128, 64, 32])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Training
history = autoencoder.fit(x_train, x_train,
                         epochs=50,
                         batch_size=256,
                         shuffle=True,
                         validation_data=(x_test, x_test))

##2. Overcomplete Autoencoder dengan Regularization
Overcomplete autoencoder memiliki latent space yang lebih besar dari input. Tanpa regularization, model ini akan belajar identity function. Oleh karena itu, perlu ditambahkan regularization seperti sparse regularization atau denoising.

In [ ]:
# Sparse Autoencoder dengan L1 regularization
def create_sparse_autoencoder(input_dim, encoding_dim, sparsity_constraint=1e-5):
    input_layer = keras.layers.Input(shape=(input_dim,))

    # Encoder dengan L1 regularization
    encoded = keras.layers.Dense(encoding_dim,
                                activation='relu',
                                activity_regularizer=keras.regularizers.l1(sparsity_constraint))(input_layer)

    # Decoder
    decoded = keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = keras.Model(input_layer, decoded)
    return autoencoder

sparse_autoencoder = create_sparse_autoencoder(784, 1000)
sparse_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

##3. Denoising Autoencoder
Denoising autoencoder dilatih untuk merekonstruksi input yang bersih dari input yang telah ditambahkan noise. Ini membantu model belajar representasi yang lebih robust dan berguna.

In [ ]:
# Implementasi Denoising Autoencoder
def add_noise(data, noise_factor=0.5):
    """Menambahkan Gaussian noise ke data"""
    noise = np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    return np.clip(data + noise_factor * noise, 0.0, 1.0)

def create_denoising_autoencoder(input_dim, encoding_dim):
    input_layer = keras.layers.Input(shape=(input_dim,))

    # Encoder
    encoded = keras.layers.Dense(encoding_dim, activation='relu')(input_layer)

    # Decoder
    decoded = keras.layers.Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = keras.Model(input_layer, decoded)
    return autoencoder

# Prepare noisy data
x_train_noisy = add_noise(x_train, 0.5)
x_test_noisy = add_noise(x_test, 0.5)

denoising_autoencoder = create_denoising_autoencoder(784, 128)
denoising_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Training dengan noisy input dan clean target
history = denoising_autoencoder.fit(x_train_noisy, x_train,
                                   epochs=50,
                                   batch_size=128,
                                   validation_data=(x_test_noisy, x_test))

##Variational Autoencoder (VAE)
Variational Autoencoder adalah pengembangan dari autoencoder tradisional yang dapat menghasilkan data baru. VAE menggunakan pendekatan probabilistik dimana encoder menghasilkan distribusi probabilitas (mean dan variance) dari latent variables, bukan nilai deterministik.
<br><br>

##Konsep Teoritis VAE
VAE berdasarkan pada prinsip variational inference dan memiliki komponen loss yang terdiri dari:

- Reconstruction Loss: Mengukur seberapa baik decoder merekonstruksi input
- KL Divergence Loss: Mengukur perbedaan antara distribusi laten yang dipelajari dengan prior distribution (biasanya standard normal)

In [ ]:
# Implementasi Variational Autoencoder
class VAE(keras.Model):
    def __init__(self, latent_dim, input_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.input_dim = input_dim

        # Encoder
        self.encoder = keras.Sequential([
            keras.layers.InputLayer(input_shape=(input_dim,)),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(latent_dim + latent_dim),  # mean dan log_var
        ])

        # Decoder
        self.decoder = keras.Sequential([
            keras.layers.InputLayer(input_shape=(latent_dim,)),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dense(input_dim, activation='sigmoid')
        ])

    def encode(self, x):
        mean_logvar = self.encoder(x)
        mean, logvar = tf.split(mean_logvar, num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        """Reparameterization trick"""
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean

    def decode(self, z):
        return self.decoder(z)

    def call(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decode(z)
        return x_recon, mean, logvar

# Loss function untuk VAE
def vae_loss(x, x_recon, mean, logvar):
    # Reconstruction loss
    recon_loss = keras.losses.binary_crossentropy(x, x_recon)
    recon_loss = tf.reduce_mean(recon_loss)

    # KL divergence loss
    kl_loss = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mean) - tf.exp(logvar))

    total_loss = recon_loss + kl_loss
    return total_loss, recon_loss, kl_loss

# Training VAE
vae = VAE(latent_dim=20, input_dim=784)
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

@tf.function
def train_step(x):
    with tf.GradientTape() as tape:
        x_recon, mean, logvar = vae(x)
        loss, recon_loss, kl_loss = vae_loss(x, x_recon, mean, logvar)

    gradients = tape.gradient(loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(gradients, vae.trainable_variables))

    return loss, recon_loss, kl_loss

# Training loop
epochs = 100
batch_size = 128

for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        batch = x_train[i:i+batch_size]
        loss, recon_loss, kl_loss = train_step(batch)

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Recon: {recon_loss:.4f}, KL: {kl_loss:.4f}')

##Aplikasi Autoencoder
###1. Dimensionality Reduction
Autoencoder dapat digunakan sebagai alternatif non-linear untuk PCA dalam dimensionality reduction.

In [ ]:
# Visualisasi hasil encoding
def visualize_latent_space(encoder, data, labels, latent_dim):
    if latent_dim == 2:
        # Jika latent dimension = 2, plot langsung
        encoded_data = encoder.predict(data)
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=labels, cmap='tab10')
        plt.colorbar(scatter)
        plt.title('Latent Space Representation')
        plt.show()
    else:
        # Jika latent dimension > 2, gunakan t-SNE
        from sklearn.manifold import TSNE
        encoded_data = encoder.predict(data)
        tsne = TSNE(n_components=2, random_state=42)
        encoded_2d = tsne.fit_transform(encoded_data)

        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(encoded_2d[:, 0], encoded_2d[:, 1], c=labels, cmap='tab10')
        plt.colorbar(scatter)
        plt.title('t-SNE of Latent Space')
        plt.show()

# Contoh penggunaan untuk visualisasi
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), 784))

# Visualisasi latent space
visualize_latent_space(encoder, x_train[:5000], y_train[:5000], 32)

###2. Anomaly Detection
Autoencoder dapat digunakan untuk mendeteksi anomali dengan mengukur reconstruction error.

In [ ]:
# Anomaly Detection menggunakan Autoencoder
def detect_anomalies(autoencoder, data, threshold_percentile=95):
    """
    Mendeteksi anomali berdasarkan reconstruction error
    """
    # Prediksi rekonstruksi
    reconstructed = autoencoder.predict(data)

    # Hitung reconstruction error
    mse = np.mean(np.power(data - reconstructed, 2), axis=1)

    # Tentukan threshold berdasarkan percentile
    threshold = np.percentile(mse, threshold_percentile)

    # Identifikasi anomali
    anomalies = mse > threshold

    return anomalies, mse, threshold

# Contoh penggunaan
anomalies, errors, threshold = detect_anomalies(autoencoder, x_test)

print(f"Jumlah anomali terdeteksi: {np.sum(anomalies)}")
print(f"Threshold: {threshold:.4f}")

# Visualisasi hasil
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(errors, bins=50, alpha=0.7)
plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold ({threshold:.4f})')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Reconstruction Errors')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(range(len(errors)), errors, c=anomalies, cmap='coolwarm', alpha=0.6)
plt.axhline(threshold, color='red', linestyle='--')
plt.xlabel('Sample Index')
plt.ylabel('Reconstruction Error')
plt.title('Anomaly Detection Results')
plt.show()

###3. Data Generation dengan VAE
VAE dapat menghasilkan data baru dengan sampling dari latent space.

In [ ]:
# Generate new data menggunakan VAE
def generate_new_samples(vae, num_samples=10):
    """Generate new samples from VAE"""
    # Sample dari standard normal distribution
    z = tf.random.normal(shape=(num_samples, vae.latent_dim))

    # Decode untuk mendapatkan generated samples
    generated_samples = vae.decode(z)

    return generated_samples.numpy()

# Generate dan visualisasi new samples
new_samples = generate_new_samples(vae, 25)

# Reshape untuk visualisasi (28x28 untuk MNIST)
new_samples = new_samples.reshape(-1, 28, 28)

# Plot generated samples
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(new_samples[i], cmap='gray')
    plt.axis('off')
plt.suptitle('Generated Samples from VAE')
plt.show()

##Convolutional Autoencoder
Untuk data image, Convolutional Autoencoder lebih efektif karena dapat mempertahankan spatial information.

In [ ]:
# Implementasi Convolutional Autoencoder
def create_conv_autoencoder(input_shape):
    # Encoder
    input_layer = keras.layers.Input(shape=input_shape)

    # Encoder layers
    x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = keras.layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder layers
    x = keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = keras.layers.UpSampling2D((2, 2))(x)
    x = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.UpSampling2D((2, 2))(x)
    x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.UpSampling2D((2, 2))(x)
    decoded = keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = keras.Model(input_layer, decoded)
    encoder = keras.Model(input_layer, encoded)

    return autoencoder, encoder

# Prepare data untuk convolutional autoencoder
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Create dan compile model
conv_autoencoder, conv_encoder = create_conv_autoencoder((28, 28, 1))
conv_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Training
history = conv_autoencoder.fit(x_train, x_train,
                              epochs=50,
                              batch_size=128,
                              validation_data=(x_test, x_test))

##Tips dan Best Practices
###1. Hyperparameter Tuning


In [ ]:
# Contoh hyperparameter tuning untuk autoencoder
def tune_autoencoder_hyperparams():
    # Parameter yang bisa di-tune
    encoding_dims = [16, 32, 64, 128]
    learning_rates = [0.001, 0.01, 0.1]
    batch_sizes = [64, 128, 256]

    best_loss = float('inf')
    best_params = {}

    for encoding_dim in encoding_dims:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                # Create model
                autoencoder, _ = create_simple_autoencoder(784, encoding_dim)
                autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                                  loss='binary_crossentropy')

                # Train dengan early stopping
                early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                              patience=5,
                                                              restore_best_weights=True)

                history = autoencoder.fit(x_train, x_train,
                                        epochs=20,
                                        batch_size=batch_size,
                                        validation_data=(x_test, x_test),
                                        callbacks=[early_stopping],
                                        verbose=0)

                # Evaluasi
                val_loss = min(history.history['val_loss'])

                if val_loss < best_loss:
                    best_loss = val_loss
                    best_params = {
                        'encoding_dim': encoding_dim,
                        'learning_rate': lr,
                        'batch_size': batch_size,
                        'val_loss': val_loss
                    }

    return best_params

# Jalankan hyperparameter tuning
# best_params = tune_autoencoder_hyperparams()
# print("Best parameters:", best_params)

###2. Monitoring Training

In [ ]:
# Callback untuk monitoring training
class AutoencoderMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data, encoder, decoder):
        self.validation_data = validation_data
        self.encoder = encoder
        self.decoder = decoder

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
            # Visualisasi rekonstruksi setiap 10 epoch
            x_test_sample = self.validation_data[0][:5]
            encoded_imgs = self.encoder.predict(x_test_sample)
            decoded_imgs = self.decoder.predict(encoded_imgs)

            plt.figure(figsize=(15, 4))
            for i in range(5):
                # Original
                plt.subplot(2, 5, i + 1)
                plt.imshow(x_test_sample[i].reshape(28, 28), cmap='gray')
                plt.title(f'Original {i+1}')
                plt.axis('off')

                # Reconstructed
                plt.subplot(2, 5, i + 1 + 5)
                plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
                plt.title(f'Reconstructed {i+1}')
                plt.axis('off')

            plt.suptitle(f'Epoch {epoch}')
            plt.show()

# Penggunaan monitor
monitor = AutoencoderMonitor((x_test, x_test), encoder, decoder)

##Kesimpulan
Autoencoder adalah tool yang sangat powerful dalam machine learning dengan berbagai aplikasi praktis. Konsep-konsep utama yang perlu dipahami:

- Architecture: Encoder-decoder structure dengan bottleneck
- Loss Function: Reconstruction loss sebagai objektif utama
- Variants: Undercomplete, overcomplete, sparse, denoising, dan variational
- Applications: Dimensionality reduction, feature learning, anomaly detection, dan data generation
<br><br>

Pemahaman yang mendalam tentang autoencoder memungkinkan kita untuk:

- Melakukan unsupervised learning yang efektif
- Mengekstrak representasi berguna dari data
- Mendeteksi anomali dalam data
- Menghasilkan data sintesis yang realistis

Implementasi yang tepat memerlukan pemahaman tentang arsitektur yang sesuai dengan jenis data, tuning hyperparameter yang cermat, dan evaluasi yang komprehensif terhadap kualitas representasi yang dihasilkan.